In [1]:
import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
data = pd.read_csv(r'vect_fv.csv',dtype={'error_label':object} )
#print(data)
data.head()

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,l9,l10,l11,l12,l13,l14,l15,l16,l17,l18
0,10001,0.41463,1.18500,1.12590,-0.63784,-1.60940,-0.19869,-0.23010,-0.69190,-0.99791,...,0,0,0,0,0,0,0,1,0,0
1,10002,-1.68950,-0.63024,-0.23417,-2.21810,-0.36157,-0.52164,-0.31883,1.06740,0.75016,...,0,0,0,0,0,0,0,0,0,0
2,10003,-1.71670,2.40120,0.44570,1.14400,-0.53560,-0.69040,0.39990,1.63330,-0.43470,...,0,0,0,0,0,0,0,0,0,0
3,10004,-0.07260,0.31790,-1.38690,1.83000,-1.08930,-1.15900,0.05750,0.72450,-0.47780,...,0,0,0,0,0,0,0,0,0,0
4,10005,0.32088,-1.08450,-1.15780,-0.63845,1.03060,0.38707,0.95113,0.30132,-1.08750,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# split labels and return new dataframe
#Create independent and Dependent Features
#data=split_label(data)
def split_label_dict(df):
    # Change 'label' to whatever naming used in the original DataFrame.
    temp_df = df['error_label'].str.split("", n = -1, expand = True)
    for i in range(1, 19):
        df[f'l{6}'] = temp_df[6]
        df[f'l{6}'] = pd.to_numeric(df[f'l{6}'])
    return df

newdata=split_label_dict(data)
#print(newdata)
#Create independent and Dependent Features
df = newdata.values[:, 1:4040]
X = pd.DataFrame(df) 
Y = newdata["l6"]
#print(X)
#print(Y)
#X.head()
Y.head()

In [ ]:
df = data.values[:, 1:4040]
X = pd.DataFrame(df) 
Y = data["l17"]
#print(X)
#print(Y)
X.head()
#Y.head()

In [7]:
df = pd.DataFrame(data)
#print df
new_df = df.drop(['error_label','l18','l17','l16','l15','l14','l13','l12','l11','l10','l9','l8','l7','l6','l5','l3','l4','l2','l1', 'ID'],axis=1)
Y = data["l14"]
X=new_df
print(new_df.shape)
print(X.shape)
#print(new_df)
print(Y.shape)

(62183, 4040)
(62183, 4040)
(62183,)


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X, Y)
print(X_resampled.shape, y_resampled.shape )

In [ ]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

In [ ]:
def prog(X, Y, percent=0.8):
    X["Label"]=Y
    print(Y)
    size0, size1=X[( (X["Label"]==0) )].shape[0], X[( (X["Label"]==1) )].shape[0]
    sz=[size0, size1]; Label=[0, 1]
    mn=min(sz); index=sz.index(mn)
    mn8=int(percent*mn)
    
    tr_labelA=X[( (X["Label"]==Label[index]) )][:mn8]
    tr_labelB=X[( (X["Label"]==Label[(index+1)%2]) )][:mn8]
    ts_labelA=X[( (X["Label"]==Label[index]) )][mn8:] 
    ts_labelB=X[( (X["Label"]==Label[(index+1)%2]) )][mn8:]
    X=pd.concat([tr_labelA, tr_labelB])
    
    Xts=pd.concat([ts_labelA, ts_labelB])
    
    Y=X["Label"]
    Yts=Xts["Label"]
    del X["Label"]
    del Xts["Label"]
    return X, Xts, Y, Yts

X_train, X_test, y_train, y_test=prog(X, Y, percent=0.8)


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=69,stratify=Y)


In [ ]:
accuracy_rate = []

# Will take some time
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    score=cross_val_score(knn,X,Y,cv=10)
    accuracy_rate.append(score.mean())
print(accuracy_rate)

In [9]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Fitting the classifier into the Training set
model = KNeighborsClassifier(n_neighbors=30)
model.fit(X_train,y_train)
# Predicting the test set results
y_predict = model.predict(X_test)
print(accuracy_score(y_test,y_predict))
pd.crosstab(y_test,y_predict)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pickle
results=classification_report(y_test,y_predict)
macro=f1_score(y_test, y_predict, average='macro')
micro=f1_score(y_test, y_predict, average='micro')
weighted=f1_score(y_test, y_predict, average='weighted')
avg=f1_score(y_test, y_predict)
#accuracy_score=accuracy_score(y_test,y_predict)
cm = confusion_matrix(y_test, y_predict)
roc=roc_auc_score(y_test,y_predict)
print(results)
print(macro)
print(micro)
print(roc)
print(cm)

In [ ]:
#plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
  #       markerfacecolor='red', markersize=10)
plt.plot(range(1,40),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from collections import Counter
smk = SMOTE()
#smk = SMOTETomek()
X_res,y_res=smk.fit_sample(X_train.astype('float'),y_train)
print(X_res.shape,y_res.shape)
#print(X_res)
#print(y_res)
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Fitting the classifier into the Training set
model = KNeighborsClassifier()
model.fit(X_res,y_res)
# Predicting the Training set results
#y_train_predict=model.predict(X_res)
# Predicting the test set results
y_predict = model.predict(X_test)
#print(accuracy_score(y_res,y_train_predict))
print(accuracy_score(y_test,y_predict))
pd.crosstab(y_test,y_predict)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_test, y_test)
print(X_resampled.shape, y_resampled.shape )
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_test)))
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

In [ ]:
# Predicting the test set results
y_predict = model.predict(X_resampled)
print(accuracy_score(y_resampled,y_predict))
pd.crosstab(y_resampled,y_predict)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
print(classification_report(y_resampled,y_predict))
print(accuracy_score(y_resampled,y_predict))
roc=roc_auc_score(y_resampled,y_predict)
print(roc)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pickle
results=classification_report(y_test,y_predict)
macro=f1_score(y_test, y_predict, average='macro')
micro=f1_score(y_test, y_predict, average='micro')
weighted=f1_score(y_test, y_predict, average='weighted')
avg=f1_score(y_test, y_predict, average=None)

accuracy_score=accuracy_score(y_test,y_predict)
cm = confusion_matrix(y_test, y_predict)
roc=roc_auc_score(y_test,y_predict)

print(cm)
print(results) 
print(roc)
#print(accuracy_score)
